In [35]:
import jax
import jax.numpy as jnp
import numpy as np
import soundfile as sf
from IPython.display import Audio
from matplotlib import pyplot as plt

from vkplatejax.coupling import (
    compute_coupling_matrix,
)
from vkplatejax.excitations import create_1d_raised_cosine
from vkplatejax.ftm import (
    PlateParameters,
    damping_term_simple,
    eigenvalues_from_pde,
    evaluate_rectangular_eigenfunctions,
    plate_eigenvalues,
    plate_wavenumbers,
    stiffness_term,
)
from vkplatejax.sv import (
    A_inv_vector,
    B_vector,
    C_vector,
    solve_sv_vk_jax_scan,
)

jax.config.update("jax_enable_x64", True)

In [36]:
n_max_modes_x = 30
n_max_modes_y = 30
sampling_period = 1 / 44100
n_phi = 30
n_psi = 30

params = PlateParameters(
    h=0.0005,
    rho=7850,
    E=2e12,
    nu=0.3,
    d1=0.05,
    d3=0.1,
    Ts0=0,
    l1=0.2,
    l2=0.3,
)
wnx, wny = plate_wavenumbers(
    n_max_modes_x,
    n_max_modes_y,
    params.l1,
    params.l2,
)
lambda_mu = plate_eigenvalues(wnx, wny)

In [37]:
eigvals = eigenvalues_from_pde(
    params,
    lambda_mu,
).reshape(-1)
exact_freq = eigvals.imag / (2 * jnp.pi)

indices = np.argsort(lambda_mu.ravel())[:n_phi]
ky_indices, kx_indices = np.unravel_index(indices, lambda_mu.shape)
ky_indices, kx_indices = ky_indices + 1, kx_indices + 1
# note that these are switched and 1 added
selected_indices = np.stack([kx_indices, ky_indices], axis=-1)
lambda_mu = lambda_mu.reshape(-1).sort()[:n_phi]

In [38]:
k = stiffness_term(params, lambda_mu)
c = damping_term_simple(np.sqrt(k))
print(f"omega_mu = {np.sqrt(k)}")

omega_mu = [  860.75479665  1655.29768587  2648.4762974   2979.53583457
  3443.01918662  4767.25733531  4833.46924275  5628.01213197
  6422.55502119  6621.19074349  7217.09791041  7746.79316989
  9004.81941115  9600.72657806  9799.36230037 10130.42183754
 10593.90518959 11918.14333829 11918.14333829 11984.35524572
 13573.44102416 13772.07674646 14897.67917286 15162.5268026
 15361.1625249  15957.06969182 16155.70541412 17281.30784051
 17546.15547025 18340.69835947]


In [39]:
A_inv = A_inv_vector(sampling_period, c * 2)
B = B_vector(sampling_period, k) * A_inv
C = C_vector(sampling_period, c * 2) * A_inv

In [40]:
eigenfunction_norm = 0.25 * params.l1 * params.l2
sampling_factor = (1 / params.density) * A_inv

In [ ]:
force_position = (0.05, 0.05)
mode_gains_at_pos = evaluate_rectangular_eigenfunctions(
    selected_indices,
    force_position,
    params=params,
)

# generate a 1d raised cosine excitation
rc = create_1d_raised_cosine(
    duration=2.0,
    start_time=0.010,
    end_time=0.012,
    amplitude=8000.0,
    sample_rate=44100,
)

# the modal excitation needs to be scaled by A_inv and divided by the density
mode_gains_at_pos_normalised = (mode_gains_at_pos / params.density) * A_inv
modal_excitation_normalised = rc[:, None] * mode_gains_at_pos_normalised

print(modal_excitation_normalised.max(), modal_excitation_normalised.min())

1.0476303979749512e-06 -7.406814744082009e-07


Get coupling coefficients

In [63]:
H0, H1, H2 = compute_coupling_matrix(
    n_psi,
    n_phi,
    params.l1,
    params.l2,
    kx_indices,
    ky_indices,
)

e = (params.E * eigenfunction_norm) / (2 * params.rho)

H1 = H1 * np.sqrt(e)
sampling_factor = A_inv

In [64]:
_, modal_sol = solve_sv_vk_jax_scan(
    A_inv,
    B,
    C,
    modal_excitation_normalised,
    Hv=H1,
    g=A_inv,
)

In [67]:
readout_position = (0.1, 0.1)

mode_gains_at_pos = evaluate_rectangular_eigenfunctions(
    selected_indices,
    readout_position,
    params=params,
)

out_vel = (modal_sol @ mode_gains_at_pos) / eigenfunction_norm
out_vel = np.diff(out_vel, axis=0)

display(Audio(out_vel, rate=44100))
sf.write("out_vk.wav", out_vel / np.max(out_vel), 44100)